# Multilayer Perceptron


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Multilayer Perceptron is a type of artificial neural network that consists of multiple layers of interconnected nodes, called neurons. It is a feedforward neural network, meaning that the information flows in one direction, from the input layer to the output layer.

MLPs are widely used in various machine learning tasks, such as classification, regression, and pattern recognition. It is known for its ability to learn complex patterns and make accurate predictions.

The MLP uses a supervised learning technique called backpropagation for training. In backpropagation, the error is calculated between the predicted and actual output, and this error is propagated back through the network, starting from the output layer to the hidden layer(s), to adjust the weights for the next iteration.

In [ ]:
# Plot the decision boundary of a classifier, you can ignore this function
def plotgrid(f, dr=0):
    result = np.zeros((100, 100))
    for i in range(100):
        for j in range(100):
            x = np.array([i/100.0, j/100.0])
            z = f(x)
            if dr == 0:
                z /= sum(z)
                result[i, j] = z[1]
            else:
                result[i, j] = np.argmax(z)
    return result.T[::-1]

## Fully Connected Feed-Forward Network

The layers of an MLP typically include:

1. Input Layer: This is where the model receives its input features. Each node in this layer represents one feature.
2. Hidden Layer(s): These are layers between the input and output layers where the actual processing happens via a system of weighted 'connections'. The most important aspect of the MLP is that these layers are "hidden", meaning their values are not observable in the training set. Each node in these layers uses a nonlinear activation function, which allows the MLP to capture complex patterns.
3. Output Layer: This layer produces the final predictions of the model. The activation function used in this layer depends on the nature of the problem. For example, for a binary classification problem (our problem), a sigmoid function can be used.


### Mathematics Background

In a neural network, each layer consists of multiple neurons, and each neuron is connected to all neurons from the previous layer. The input to a neuron is a weighted sum of the outputs of the neurons from the previous layer. If we consider all neurons in a layer, this operation can be represented as a matrix multiplication. The matrix contains the weights of the connections between the neurons in the current layer and the previous layer. The input data is multiplied by this weight matrix to give the initial output for each neuron in the current layer.

$$ y = f(X * W) $$

Real-world data is often complex and non-linear. Activation functions introduce non-linearity into the network, allowing it to learn and model these complex patterns. Without an activation function, no matter how many layers the network has, it would still behave like a single-layer model because the composition of linear functions is still a linear function.

In our example, three types of activation functions are depicted:
1. Identity: This is a linear function where the output is the same as the input. It is used when we want to model a linear relationship. `activation==None`
2. ReLU: This function outputs the input directly if it's positive; otherwise, it outputs zero. It's widely used in deep learning models because it helps the model to train faster without sacrificing much accuracy.
3. Sigmoid: This function maps the input values into a range between 0 and 1, making it useful for outputting probabilities in binary classification problems.

We will look at them later separately!

In [ ]:
class FCLayer:
    def __init__(self, input_size, output_size, activation=None):
        self.relu = activation == 'relu'
        self.sigmoid = activation == 'sigmoid'
        self.weights = np.random.randn(
            input_size, output_size) / np.sqrt(input_size)
        self.weight_update = np.zeros_like(self.weights)

    def forward(self, input):
        self.input = input.copy()
        self.y = np.dot(self.input, self.weights)
        if self.relu:
            self.y[self.y < 0] = 0
        if self.sigmoid:
            self.y = 1.0 / (1.0 + np.exp(-self.y))
        return self.y

    def backward(self, grad, _):
        if self.relu:
            grad[self.y <= 0] = 0
        if self.sigmoid:
            grad = grad * self.y * (1 - self.y)
        self.weight_update += np.outer(self.input, grad)
        return np.dot(grad, self.weights.T)

    def update_weights(self, learning_rate):
        self.weights -= learning_rate * self.weight_update
        self.weight_update = np.zeros_like(self.weights)


class Network:
    def __init__(self, topology, learning_rate):
        self.learning_rate = learning_rate
        self.topology = topology

    def update_weights(self):
        for layer in self.topology:
            layer.update_weights(self.learning_rate)

    def forward(self, x):
        for layer in self.topology:
            x = layer.forward(x)
        return x

    def backward(self, x, y):
        for layer in self.topology:
            x = layer.forward(x)
        for layer in reversed(self.topology):
            y = layer.backward(y, self.learning_rate)


network = Network(
    topology=[
        # Input Layer
        FCLayer(2, 5, activation='relu'),
        # Output Layer
        FCLayer(5, 2, activation='sigmoid')
    ],
    learning_rate=1)

## Binary Classification

We generate randomly a synthetic dataset suitable for a binary classification problem.
It consists of 100 2D samples. The optimal decision boundary is given by the equation:

$$ y = x - 0.3 $$

In [ ]:
def random_data():
    data = np.random.random_sample((100, 2))
    labels = (data[:, 0]-data[:, 1] < 0.3)

    d0 = data[labels == False]
    d1 = data[labels]

    targets = np.array([labels, 1-labels]).T
    return data, targets, labels, d0, d1

data, targets, labels, d0, d1 = random_data()
plt.plot(d0[:, 0], d0[:, 1], "bo")
plt.plot(d1[:, 0], d1[:, 1], "ro")

# Plot the decision boundary of a classifier: y = x - 0.3
x = np.linspace(0.3, 1, 2)
y = x - 0.3
plt.plot(x, y)

In [ ]:
# Before training, the decision boundary is fuzzy and random
plt.imshow(plotgrid(network.forward, dr=0))

### Training

The optimization of neural networks is called training and the algorithm is called backpropagation:

1. Forward Pass: The network makes a prediction based on the current weights. This prediction is compared to the actual output, and the difference is measured using a loss function. The goal is to minimize this loss.

2. Backward Pass (Backpropagation): In this step, the algorithm calculates how much each weight contributed to the loss, starting from the output layer and moving back towards the input layer. This is done by applying the chain rule of calculus to compute gradients (partial derivatives) of the loss function with respect to the weights and biases.

Once these gradients are known, the weights are updated in a way that reduces the loss. This is typically done using an optimization algorithm like gradient descent.

In simple terms, backpropagation is like a student learning from their mistakes. After answering a question (forward pass), the student checks their answer against the correct one. If the answer is wrong, the student reviews their solution to understand where they made a mistake (backward pass) and then corrects it. This process is repeated until the student is able to answer the question correctly.

In our example, we optimize the Mean Squared Error

$$ MSE = \frac{1}{2}(y - y_t)^2  $$


In [ ]:
# Training
for e in range(1000):
    i = np.random.randint(0, len(data))
    out = network.forward(data[i])
    # We divide by 2 to make the derivative of the loss easier
    # Loss = 1/2(target - output)^2
    # dLoss/dOutput = -(target - output)
    network.backward(data[i], -(targets[i]-out))
    network.update_weights()

correct = 0
for i in range(len(data)):
    out = np.argmax(network.forward(data[i]))
    if out == np.argmax(targets[i]):
        correct += 1

print("Correct predicted: ", 1.0 * correct /len(data))
plt.imshow(plotgrid(network.forward, dr=0))

Toy around with the parameters. See how the network behaves, if you adapt the activation functions.
Check what happens when you tweak the number of epochs.

## Activation Functions

In [ ]:
def identity(x):
    return x

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

x = np.linspace(-10, 10, 100)

_, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].plot(x, identity(x))
axs[0].set_title('Identity')

axs[1].plot(x, relu(x))
axs[1].set_title('ReLu')

axs[2].plot(x, sigmoid(x))
axs[2].set_title('Sigmoid')